In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vedant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vedant\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
#Reading the excel data
#df = pd.read_excel(r"C:\College\sem6\NLP\project\text_dataset.xlsx")
df = pd.read_excel(r"C:\Users\Vedant\OneDrive\Desktop\College\TY\SEM 6\NLP\preprocessed_text.xlsx")
df1 = pd.read_csv(r"C:\Users\Vedant\Downloads\mail_data.csv")
df = pd.concat([df,df1])
df

,Message
0,In The Simpsons Movie released in July 2007 na...
1,professional logo for you now working on you...
2,"get your babies diapers bill paid for , for a..."
3,very uuseful attache how to save on your med...
4,"model effort in houston christian , our spr..."
...,...
5567,This is the 2nd time we have tried 2 contact u...
5568,Will ü b going to esplanade fr home?
5569,"Pity, * was in mood for that. So...any other s..."
5570,The guy did some bitching but I acted like i'd...


# EDA

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8362 entries, 0 to 5571
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  8362 non-null   object
dtypes: object(1)
memory usage: 130.7+ KB


In [11]:
# check for missing values and remove
print(df.isnull().sum())

Message    0
dtype: int64


In [12]:
# check for duplicate values and remove
print(df.duplicated().sum())
df = df.drop_duplicates(keep='first')
df.shape

1071


(7291, 1)

# Preprocessing

In [14]:
# replace unnecessary letters/words
replace_list = [" &lt;#&gt;","Ã","&gt;","\\n","\\t","\\r","â","€","™","ð","Ÿ","ðŸ‘","$","â€™ll","ƒ","¢","â€ƒ","â€¢","Â§","§","Â","Ã¼","Ã","¼","º","œ","˜","£","â€“","â€œ","&lt;#&gt;","â€Œ","ðŸŽ","ð","Ÿ","Ž","Í","â€Œ ï»¿ Í","ï»¿","ðŸ”¨","ðŸ¤©","©","¤","±","ðŸ˜±","ðŸ˜±"," Í â€Œ ï»¿","ÿ","x = = x","*"]
for letter in replace_list:
    df['Message'] = df['Message'].str.replace(letter, '')

C:\Users\Vedant\AppData\Local\Temp\ipykernel_17736\221389072.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Message'] = df['Message'].str.replace(letter, '')


In [16]:
email_replace_list = ["Subject:","cc:"]
for letter in email_replace_list:
    df['Message'] = df['Message'].str.replace(letter, '')

C:\Users\Vedant\AppData\Local\Temp\ipykernel_17736\2984804244.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Message'] = df['Message'].str.replace(letter, '')


In [17]:
lemmatizer = WordNetLemmatizer()

def preprocessing_text(text):
    if isinstance(text, str):
        text = nltk.word_tokenize(text)

        filtered_words = []
        for word in text:
            if word.isalnum():
                filtered_words.append(word)

        text = filtered_words[:]
        
        lemmatized_words= []
        for word, pos in nltk.pos_tag(text):
            if word.isalnum() and word not in stopwords.words('english') and word not in string.punctuation:
                # Get the first letter of the POS tag
                pos = pos[0].lower()
                # Consider only adjectives, nouns, and verbs for lemmatization
                if pos in {'a', 'n', 'v'}:
                    lemmatized_words.append(lemmatizer.lemmatize(word, pos))
                else:
                    lemmatized_words.append(word)

        return " ".join(lemmatized_words)
    else:
        # Convert non-string types to string
        return str(text)

# Apply the modified transform_text2 function using lambda to the 'Text' column
df['preprocessed_text'] = df['Message'].apply(lambda x: preprocessing_text(x))
df['preprocessed_text']

C:\Users\Vedant\AppData\Local\Temp\ipykernel_17736\1090032615.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed_text'] = df['Message'].apply(lambda x: preprocessing_text(x))


0       In The Simpsons Movie release July 2007 name b...
1       professional logo work company image start vis...
2       get baby diaper bill pay year family could def...
3       uuseful attache save medlcations 70 pharmsho f...
4       model effort houston christian spring fall win...
                              ...                        
5567    This 2nd time try 2 contact U 750 Pound prize ...
5568                        Will ü b go esplanade fr home
5569                              Pity mood So suggestion
5570    The guy bitching I act like interested buy som...
5571                                   Rofl Its true name
Name: preprocessed_text, Length: 7291, dtype: object

ZERO-SHOT CLASSIFICATION

In [19]:
import pandas as pd
from transformers import pipeline

# Define the classes
classes_verbalized = ["politics", "economy", "entertainment", "environment", "sports", "finance","spam","personal"]

# Initialize the zero-shot classification pipeline
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33")

# Create empty lists to store results
predicted_categories = []

# Iterate over each text in the DataFrame
for text in df["preprocessed_text"]:
    # Perform zero-shot classification
    output = zeroshot_classifier(text, classes_verbalized, multi_label=False)
    # Get the index of the label with the highest score
    max_score_index = output["scores"].index(max(output["scores"]))
    # Get the label with the highest score
    predicted_label = output["labels"][max_score_index]
    
    # Append the predicted category to the list
    predicted_categories.append(predicted_label)

# Add the predicted categories as a new column to the DataFrame
df["category"] = predicted_categories

In [ ]:
print(df)